In [1]:
using GPFlux
using Flux
using Zygote
using Random; Random.seed!(5);

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/hongbinren/.julia/packages/CUDAdrv/aBgcd/src/CUDAdrv.jl:69


# Implementation of Gaussian process layer
Traditional neural network layer is writen as:
$$y=f(Wx+b)$$
where $f$ is a nonlinear activation function, $W$ and $b$ are parameters. We can view this as a parametric function, another feature of this function is that it's deterministic.

Since neural network layer is nothing but a parameterised nonlinear function, we are free to substitute it with Gaussian process. In Gaussian processes, the nonlinearity is obtained by directly model the distribution of function values $f$ as a multivariate Gaussian distribution, which is specified by it's mean and covariance function:
$$
\begin{gather}
f\sim \mathcal{GP}\left(\mu(x), K(x,x')\right)\\
y\sim \mathcal{N}(0,\epsilon^2)
\end{gather}
$$

Compared to the normal neural network function, the Gaussian process approach has several advantages:
* Non-parametric, it's realy parameter efficient ( the number of hyperparameters is significantly lower than the number of weights and biases in a common fully connected neural network ) and therefore isn't adapt to overfitting.
* Enable us to deal with uncertainty
* Allow us to explore the infinite-width limit of a neural network ( which by it's nature should be more expressive than finite width layer neural network )

With a Gaussian process layer, it's feasible to implement several modern GP-related architecture:
* GPLVM
* Deep GP

In [2]:
# example data
X = rand(3, 5);

In [3]:
# specify a Gaussian process with zero mean and isotropic square exponential kernel
zero_mean = ConstantMean([0.0])
iso_se_kernel = IsoGaussKernel([0.1], [1.0])  # length scale and overal scaling factor is in log scale
lnoise = [-0.01]                              # ϵ is also in log scale
gp = GaussProcess(zero_mean, iso_se_kernel, lnoise)    # build a Gaussian process

GaussProcess(ConstantMean([0.0]), IsoGaussKernel(ll=0.1, lσ=1.0), lnoise=-0.01)

To make Gaussian process a normal layer, we need a multi-output Gaussian process, this is down by assuming that each dimension of output is a independent draw from the Gaussian process prior ( this is a common approach taken in GPLVM and deep GP ):
$$
\begin{gather}
p(\boldsymbol{y}|\boldsymbol{X})=\Pi_{i=1}^d\,p(y_i|\boldsymbol{X})\\
y_i\sim\mathcal{GP}\left(\mu(\boldsymbol{X}), K(\boldsymbol{X, X})\right)
\end{gather}
$$
where $d$ is the dimension of the output.

In [4]:
"""
Implementation of a GP layer:
    GP layer is specified by a Gaussian process and the dimension of the output

here I write a simplified version of MvNormal, which is compatible with Distribution.jl, and use reparametrization
trick ( Kingma, Welling., Auto-Encoding Variational Bayes, arXiv:1312.6114 ) to allow backpropogation through sampling process.
"""

struct GPLayer{GPT<:GaussProcess, T<:Int}
    gp::GPT
    odims::T
end
Flux.@functor GPLayer
function (gpl::GPLayer)(x)
    xo = rand(MvNormal(gpl.gp, x), gpl.odims) # MvNormal{<:GaussProcess, <:AbstractArray} <: ContinuousMultivariateDistribution
    transpose(xo)
end

model = Chain(Dense(3, 10), GPLayer(gp, 2), Dense(2, 1)) |> f64
ps = params(model)

Params([[-0.005967428907752037 0.5589533448219299 -0.4503887891769409; 0.4754835069179535 0.5715048313140869 0.4152350127696991; … ; 0.37344714999198914 -0.10880734026432037 0.4539564251899719; -0.5453869104385376 -0.525185227394104 -0.028199106454849243], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0], [0.1], [1.0], [-0.01], [1.3460205793380737 0.7891872525215149], [0.0]])

In [5]:
# forawrd pass
model(X)

1×5 Array{Float64,2}:
 6.06098  -1.07062  4.78621  4.83658  -1.30697

In [6]:
# evaluate the derivative w.r.t to all the parameters
gs = gradient(()->sum(model(X)), ps)
gs.grads

IdDict{Any,Any} with 8 entries:
  [1.0]                     => [12.8382]
  [0.0]                     => [10.676]
  [-0.00596743 0.558953 -0… => [0.484082 -1.28918 1.26373; 1.01697 0.0246278 -0…
  [0.1]                     => [6.98602]
  [-0.01]                   => [3.71533]
  [0.0, 0.0, 0.0, 0.0, 0.0… => [-2.22045e-16, 2.22045e-16, 8.88178e-16, -8.8817…
  [1.34602 0.789187]        => [7.65121 7.92573]
  [0.0]                     => [5.0]

In [7]:
# we can also write deep GP
zero_mean1 = ConstantMean()
se_kernel1 = IsoGaussKernel([4.0], [1.0])
lnoise1 = [-0.1]
gp1 = GaussProcess(zero_mean1, se_kernel1, lnoise1)

zero_mean2 = ConstantMean()
per_kernel2 = IsoPeriodKernel([0.0], [0.0], [1.0])
lnoise2 = [-0.1]
gp2 = GaussProcess(zero_mean2, per_kernel2, lnoise2)

DeepGP = Chain(GPLayer(gp1, 2), GPLayer(gp2, 5))
ps = params(DeepGP)

Params([[0.0], [4.0], [1.0], [-0.1], [0.0], [0.0], [0.0], [1.0], [-0.1]])

In [12]:
# forward pass
"""
Warning: may throw positive definite error when evaluating cholesky factorization, this will be studied
carefully later
"""
DeepGP(X)

5×5 LinearAlgebra.Transpose{Float64,Array{Float64,2}}:
  2.0988     -1.93274    4.03467   -4.8294     3.17422
  0.0968363   3.13354    0.271508  -0.630904   1.54851
 -0.247104    1.38856   -2.38596    3.72538   -1.04662
 -3.88221    -1.61955    2.77025    1.52945   -2.82033
  1.51003     0.921324  -1.70876   -3.51345   -1.36152

In [10]:
# evaluate the derivative w.r.t to all the parameters
gs = gradient(()->sum(DeepGP(X)), ps)
gs.grads

IdDict{Any,Any} with 9 entries:
  [0.0]  => [-5.32907e-15]
  [1.0]  => [-2.47095]
  [4.0]  => [-0.0433978]
  [-0.1] => [108.323]
  [0.0]  => [25.0]
  [0.0]  => [-105.852]
  [1.0]  => [-1.64464]
  [-0.1] => [2.51644]
  [0.0]  => [-20.7026]

## Something we could discuss
* Implement training algorithm based on variational inference
* How to validate the gradient for a stochastic function ( I'm wondering if finite difference method is still a good validate method as it is for deterministic function, currently I find there is a large discrepency between the derivative evaluated by backprop and finite difference for stochastic function, maybe I'm wrong, but we can figure this out later )

# References
[1] Andreas C. Damianou, Neil D. Lawrence, Deep Gaussian Processes, 2013

[2] Diederik P. Kingma, Max Welling, Auto-Encoding Variational Bayes, 2014

[3] Michalis K. Titsias1, Neil D. Lawrence, Bayesian Gaussian Process Latent Variable Model, 2010

[4] Danilo J. Rezende, Shakir Mohamed, Daan Wierstra, Stochastic Backpropogation and Approximate Inference in Deep Generative Models, 2014